In [25]:
import cv2
import numpy as np
import sys
import os
from tqdm import tqdm

In [26]:
dir = "Thumbnail"
# If thumbnail folder doesn't exist, create it
if not os.path.exists(dir):
    os.makedirs(dir)

In [158]:
filename = "View_From_A_Blue_Moon_Trailer-576p.mp4"
resolution = 240
framerate = 15
x = 5
y = 5

In [159]:
vidname = filename[0:filename.rindex('.')]
vidcap = cv2.VideoCapture(filename)

In [160]:
frame_count = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)
fps = vidcap.get(cv2.CAP_PROP_FPS)
duration = int(frame_count/fps*1000)
print("Frame Count:", frame_count)
print("FPS:", fps)
print("Duration:", duration, "milisecond(s)")

Frame Count: 4389.0
FPS: 23.976025067483214
Duration: 183057 milisecond(s)


In [161]:
def captureFrame(vidcap, mil):
  vidcap.set(cv2.CAP_PROP_POS_MSEC, mil)
  _, image = vidcap.read()
  return image

def getHour(mil):
    return int(mil/1000/60/60)

def getMin(mil):
    return int((mil/1000/60)%60)

def getSec(mil):
    return int((mil/1000)%60)

def printTime(mil):
    return "{:02d}:{:02d}:{:02d}.{:03d}".format(
        getHour(mil),
        getMin(mil),
        getSec(mil),
        mil%1000
    )

In [162]:
def output(src,dst,img,x,y,width,height,count,f):
    f.write("\n{} --> {}\n".format(
        printTime(src), printTime(dst)
    ))

    f.write("{}#xywh={},{},{},{}\n".format(img,count%x*width,int((count%(x*y))/x)*height,width,height))

In [163]:
def generate(vidcap):
    scale = resolution / vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    width = int(scale * vidcap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(scale * vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    dim = (width,height)

    src = 0
    dst = 0
    delay = framerate * 1000
    total = int(duration/delay)
    if (duration % delay != 0):
        total = total + 1
    print("Total Images:", total)

    f = open("{}/{}.vtt".format(dir,vidname), "w")
    f.write("WEBVTT\n")

    im_h = []
    im_v = []
    img_no = 1

    for i in tqdm(range(total)):
        dst = duration if src + delay > duration else src + delay
        image = captureFrame(vidcap, src)
        image = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
        output(
            src,dst,"{}_{}.jpg".format(vidname, img_no),x,y,width,height,i,f
        )

        src = src + delay
        im_h.append(image)
        if ((i+1) % x == 0):
            image = cv2.hconcat(im_h)
            im_v.append(image)
            im_h.clear()

        if ((i+1) / x == y):
            cv2.imwrite("{}/{}_{}.jpg".format(dir,vidname, img_no),cv2.vconcat(im_v))
            im_v.clear()
            img_no = img_no + 1

    if (im_v or im_h):
        if (im_h):
            i = len(im_h)
            while(i < x):
                im_h.append(np.zeros([height, width, 3], np.uint8))
                i=i+1
            image = cv2.hconcat(im_h)
            im_v.append(image)
        
        cv2.imwrite("{}/{}_{}.jpg".format(dir,vidname, img_no),cv2.vconcat(im_v))
        im_v.clear()
    
    f.close()


In [164]:
generate(vidcap)

Total Images: 13


100%|██████████| 13/13 [00:00<00:00, 32.02it/s]
